In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Load the data
df = pd.read_csv('C:/Users/gauth/Desktop/courses/CS5604/project/dataset1/combined_data1.csv')

In [5]:
# Keep only the 'text' and 'bias' columns
df = df[['text', 'bias']]
from sklearn.preprocessing import LabelEncoder

# Assume 'df' is your DataFrame and 'bias' is the column with categorical labels
label_encoder = LabelEncoder()
df['bias_encoded'] = label_encoder.fit_transform(df['bias'])

# To see the mapping of labels to integers
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Label mapping:", label_mapping)



Label mapping: {'Center': 0, 'Lean Left': 1, 'Lean Right': 2, 'Left': 3, 'Right': 4}


In [8]:
unique_bias_values = df['bias_encoded'].unique()

# To print the unique values
print("Unique bias values:", unique_bias_values)

Unique bias values: [3 1 0 2 4]


In [16]:
import numpy as np
import re
import string
import pandas as pd
from gensim.models import Doc2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim import utils
from nltk.corpus import stopwords
import nltk

def textClean(text):
    """
    Get rid of the non-letter and non-number characters
    """
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = text.lower().split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    return (text)


def cleanup(text):
    text = textClean(text)
    text = text.translate(str.maketrans("", "", string.punctuation))
    return text


def constructLabeledSentences(data):
    sentences = []
    for index, row in data.iteritems():
        sentences.append(LabeledSentence(utils.to_unicode(row).split(), ['Text' + '_%s' % str(index)]))
    return sentences

In [18]:
# Apply cleanup function to your text data
df['clean_text'] = df['text'].apply(cleanup)

In [20]:
# Feature Extraction with TfidfVectorizer
tfidf = TfidfVectorizer(max_features=300)
X_tfidf = tfidf.fit_transform(df['clean_text']).toarray()

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, df['bias_encoded'], test_size=0.2, random_state=42)
np.save('xtr_tfidf.npy',X_train)
np.save('xte_tfidf.npy',X_test)
np.save('ytr_tfidf.npy',y_train)
np.save('yte_tfidf.npy',y_test)

NameError: name 'X_tfidf' is not defined

In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Assuming you have already loaded your dataset into x_train, x_test, y_train, y_test
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.54      0.54      0.54     35621
           1       0.47      0.53      0.50     35540
           2       0.73      0.63      0.67     35497
           3       0.47      0.48      0.47     35541
           4       0.53      0.54      0.54     35321

    accuracy                           0.54    177520
   macro avg       0.55      0.54      0.54    177520
weighted avg       0.55      0.54      0.54    177520



In [29]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.93      0.92     35621
           1       0.89      0.91      0.90     35540
           2       0.97      0.97      0.97     35497
           3       0.93      0.92      0.92     35541
           4       0.93      0.90      0.91     35321

    accuracy                           0.93    177520
   macro avg       0.93      0.93      0.93    177520
weighted avg       0.93      0.93      0.93    177520



In [5]:
import numpy as np
import re
import string
import pandas as pd
x_train = np.load('./xtr_tfidf.npy', allow_pickle=True)
x_test = np.load('./xte_tfidf.npy', allow_pickle=True)
y_train = np.load('./ytr_tfidf.npy', allow_pickle=True)
y_test = np.load('./yte_tfidf.npy', allow_pickle=True)

In [ ]:
from sklearn.svm import SVC
clf = SVC()
clf.fit(x_train, y_train)

# Predict on the evaluation data
y_pred = clf.predict(x_test)

print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gbc.fit(x_train, y_train)

y_pred = gbc.predict(x_test)

print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=200, alpha=1e-4,
                    solver='adam', verbose=10, random_state=1,
                    learning_rate_init=.01)

mlp.fit(x_train, y_train)

y_pred = mlp.predict(x_test)

print(classification_report(y_test, y_pred))